In [11]:
import pandas as pd
import re
from IPython.display import display, HTML

def calcular_ttr(texto):
    if pd.isna(texto) or texto.strip() == "": return 0
    palabras = re.findall(r'\b\w+\b', texto.lower())
    if not palabras: return 0
    palabras_unicas = set(palabras)
    return len(palabras_unicas) / len(palabras)

def crear_scorecard_enriquecido(dataframe):
    participacion_df = dataframe.groupby(['email', 'Bloque temático']).agg(
        respuestas_por_bloque=('pk_id', 'count'),
        argumentos_por_bloque=('reason', 'count')
    ).reset_index()
    participacion_df['tasa_argumentacion_bloque_%'] = \
        (participacion_df['argumentos_por_bloque'] / participacion_df['respuestas_por_bloque'] * 100).round(2)

    argumentos_df = dataframe.dropna(subset=['reason']).copy()
    argumentos_df['contiene_ejemplo'] = argumentos_df['reason'].apply(lambda x: bool(re.search(r'por ejemplo|ej\.|como es el caso|tal como|caso de', x, re.IGNORECASE)) if pd.notna(x) else False)
    argumentos_df['contiene_datos'] = argumentos_df['reason'].apply(lambda x: bool(re.search(r'\d{4}|\d+%|millones|miles|ha\b|tn\b', x, re.IGNORECASE)) if pd.notna(x) else False)
    argumentos_df['usa_lenguaje_causal'] = argumentos_df['reason'].apply(lambda x: bool(re.search(r'debido a|gracias a|por lo tanto|consecuencia|porque|ya que|dado que', x, re.IGNORECASE)) if pd.notna(x) else False)
    argumentos_df['tiene_vision_futuro'] = argumentos_df['reason'].apply(lambda x: bool(re.search(r'futuro|largo plazo|tendencia|será|permitirá|impactará|podría|debería', x, re.IGNORECASE)) if pd.notna(x) else False)
    argumentos_df['score_calidad'] = argumentos_df[['contiene_ejemplo', 'contiene_datos', 'usa_lenguaje_causal', 'tiene_vision_futuro']].sum(axis=1)

    calidad_agregada_df = argumentos_df.groupby(['email', 'Bloque temático']).agg(
        score_calidad_promedio=('score_calidad', 'mean'),
        score_calidad_total=('score_calidad', 'sum'),
        longitud_promedio_argumento=('reason', lambda x: x.str.len().mean())
    ).round(2).reset_index()

    scorecard_enriquecido = pd.merge(participacion_df, calidad_agregada_df, on=['email', 'Bloque temático'], how='left')
    scorecard_enriquecido.fillna(0, inplace=True)
    return scorecard_enriquecido

def mostrar_ranking_por_bloque_html(scorecard, nombre_categoria, top_n=5):
    header_html = f"<h3>Ranking de Participantes para el Bloque: '{nombre_categoria.split(':')[0].strip()}'</h3>"
    display(HTML(header_html))
    
    ranking_df = scorecard[scorecard['Bloque temático'] == nombre_categoria].copy()
    if ranking_df.empty:
        print("No hay datos de participación para este bloque temático.")
        return
        
    ranking_df_sorted = ranking_df.sort_values(by=['score_calidad_total', 'argumentos_por_bloque'], ascending=False)
    
    columnas_display = {
        'email': 'Email',
        'argumentos_por_bloque': 'N° Arg.',
        'score_calidad_promedio': 'Calidad Prom.',
        'score_calidad_total': 'Score Total',
        'longitud_promedio_argumento': 'Long. Prom.',
        'tasa_argumentacion_bloque_%': 'Tasa Arg. (%)'
    }
    
    display_df = ranking_df_sorted[list(columnas_display.keys())].head(top_n).rename(columns=columnas_display)
    
    # Usamos .to_html() para generar la tabla formateada
    html_output = display_df.to_html(index=False, classes='table table-striped table-bordered text-center', justify='center')
    display(HTML(html_output))

# --- Carga de Datos y Ejecución ---
nombre_archivo_excel = 'tablabase_plana3.xlsx'
try:
    df = pd.read_excel(nombre_archivo_excel)
    print(f"Archivo '{nombre_archivo_excel}' cargado exitosamente.")
    
    scorecard_enriquecido = crear_scorecard_enriquecido(df)
    print("Tabla de puntuación enriquecida creada con éxito.\n")
    
    categories = df['Bloque temático'].dropna().unique()
    for category in categories:
        mostrar_ranking_por_bloque_html(scorecard_enriquecido, category, top_n=5)
        
    # --- Ranking General ---
    header_general_html = f"<h2>Ranking General de Expertos Destacados</h2>"
    display(HTML(header_general_html))
    
    scorecard_general = scorecard_enriquecido.groupby('email').agg(
        bloques_participados=('Bloque temático', 'count'),
        total_argumentos_globales=('argumentos_por_bloque', 'sum'),
        score_calidad_total_global=('score_calidad_total', 'sum'),
        score_calidad_promedio_global=('score_calidad_promedio', 'mean')
    ).round(2).reset_index()

    scorecard_general_sorted = scorecard_general.sort_values(
        by=['score_calidad_total_global', 'total_argumentos_globales'], 
        ascending=False
    )
    
    columnas_general_display = {
        'email': 'Email',
        'bloques_participados': 'N° Bloques',
        'total_argumentos_globales': 'N° Arg. Total',
        'score_calidad_total_global': 'Score Total Global',
        'score_calidad_promedio_global': 'Calidad Prom. Global'
    }
    
    display_general_df = scorecard_general_sorted.head(15).rename(columns=columnas_general_display)
    html_general_output = display_general_df.to_html(index=False, classes='table table-striped table-bordered text-center', justify='center')
    display(HTML(html_general_output))

except FileNotFoundError:
    print(f"Error Crítico: No se encontró el archivo '{nombre_archivo_excel}'.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo 'tablabase_plana3.xlsx' cargado exitosamente.
Tabla de puntuación enriquecida creada con éxito.



Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
adriana.bocco@gmail.com,43,0.44,19.0,405.37,95.56
leonca45@gmail.com,29,0.52,15.0,279.62,60.42
rgcarri@mincyt.gov.ar,16,0.88,14.0,664.75,33.33
editho@iica.org.ar,13,0.92,12.0,555.77,86.67
lgomez@rafaela.inta.gov.ar,20,0.55,11.0,386.60,95.24


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
emilianojk@gmail.com,22,0.77,17.0,257.14,21.78
svaudagna@cnia.inta.gov.ar,22,0.45,10.0,217.68,21.15
ricardo@qinnova.com.ar,47,0.19,9.0,93.38,52.81
eortegar@uach.mx,19,0.47,9.0,252.21,19.79
smalzamora@gmail.com,16,0.50,8.0,281.81,38.10


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
santiagodelsolar@estanciaslauquen.com.ar,7,0.14,1.0,170.29,43.75
pactis@arcor.com,5,0.20,1.0,80.60,31.25
rickynegri@crea.org.ar,2,0.50,1.0,244.00,2.78
napro@inti.gob.ar,4,0.00,0.0,99.25,5.80
domanico@INTI.GOB.AR,1,0.00,0.0,249.00,100.00


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
mlloveras@pergamino.inta.gov.ar,10,0.80,8.0,295.00,22.22
ocuello@cabargentina.com,9,0.33,3.0,165.33,20.45
dpapot@minagri.gob.ar,4,0.50,2.0,276.75,9.09
rhrodriguez@correo.inta.gob.ar,3,0.00,0.0,56.67,3.75
pmilla@minagri.gob.ar,2,0.00,0.0,69.00,2.74


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
mtorres@sanjuan.inta.gov.ar,15,0.73,11.0,426.73,20.83
mlopez@aceiteselia.com.ar,6,1.00,6.0,785.50,100.00
marialourdestoujas@hotmail.com,5,0.60,3.0,254.20,13.51
mzuccardi@familiazuccardi.com,4,0.50,2.0,141.00,12.50
spicardi@criba.edu.ar,1,2.00,2.0,561.00,14.29


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
aligucea@inti.gob.ar,36,0.11,4.0,117.17,50.00
mpirovan@fiq.unl.edu.ar,16,0.25,4.0,205.88,23.53
smalzamora@gmail.com,8,0.38,3.0,259.62,36.36
ademichelis@bariloche.inta.gov.ar,11,0.09,1.0,166.64,37.93
doloresraffo@correo.inta.gov.ar,1,1.00,1.0,78.00,1.39


Email,N° Arg.,Calidad Prom.,Score Total,Long. Prom.,Tasa Arg. (%)
$email,15,0.80,12.0,380.67,37.50
speranza@inti.gob.ar,20,0.45,9.0,215.35,31.25
omvasekk@yahoo.com.ar,20,0.20,4.0,347.70,100.00
eramos@inti.gob.ar,3,0.67,2.0,136.67,3.41
ricardo@qinnova.com.ar,18,0.06,1.0,80.17,22.50


Email,N° Bloques,N° Arg. Total,Score Total Global,Calidad Prom. Global
emilianojk@gmail.com,2,33,24.0,0.70
adriana.bocco@gmail.com,1,43,19.0,0.44
leonca45@gmail.com,1,29,15.0,0.52
spicardi@criba.edu.ar,3,16,15.0,1.22
rickynegri@crea.org.ar,4,22,14.0,0.44
rgcarri@mincyt.gov.ar,1,16,14.0,0.88
ricardo@qinnova.com.ar,3,71,12.0,0.19
eortegar@uach.mx,2,28,12.0,0.40
$email,5,16,12.0,0.16
editho@iica.org.ar,1,13,12.0,0.92
